In [1]:
import sys
from dotenv import load_dotenv
import os 

load_dotenv(override=True)

sys.path.append('src')

In [2]:
#Load environment variables
from dotenv import load_dotenv
import os 

load_dotenv(override=True)

True

## 1. Extracting facts from uploaded PDF documents

In [3]:
from openai import AzureOpenAI
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient

# create AOAI client
aoai_client = AzureOpenAI(azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                     api_key=os.getenv('AZURE_OPENAI_KEY'),
                     api_version='2024-10-21')

# create Azure Storage Container Client to download the source documents
storage_account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME")
storage_container_name = "wise-inputs"

blob_service_client = BlobServiceClient(
    account_url=f"https://{storage_account_name}.blob.core.windows.net",
    credential=DefaultAzureCredential()
)

container_client = blob_service_client.get_container_client(storage_container_name)


# create Azure Document Intelligence Client to parse the PDF documents
doc_intelligence_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")


document_analysis_client = DocumentIntelligenceClient(
    endpoint=doc_intelligence_endpoint,
    credential=AzureKeyCredential(doc_intelligence_key)
)

In [4]:
from wise.fact_extractor import FactExtractor

extractor = FactExtractor(aoai_client, 
                          document_analysis_client, 
                          container_client)

facts = extractor.extract_facts()

Analyzing fidelity1.pdf...
Summarizing fidelity1.pdf...


100%|██████████| 8/8 [00:17<00:00,  2.20s/it]


Analyzing fidelity2.pdf...
Summarizing fidelity2.pdf...


100%|██████████| 11/11 [00:31<00:00,  2.84s/it]


Analyzing fidelity3.pdf...
Summarizing fidelity3.pdf...


100%|██████████| 11/11 [00:36<00:00,  3.33s/it]


Analyzing fidelity4.pdf...
Summarizing fidelity4.pdf...


100%|██████████| 11/11 [00:32<00:00,  2.93s/it]


Analyzing fidelity5.pdf...
Summarizing fidelity5.pdf...


100%|██████████| 9/9 [00:18<00:00,  2.05s/it]


In [5]:
print(facts[0])

The Fidelity International High Dividend ETF (FIDI) is listed on the NYSE Arca, Inc. and aims to provide investment returns that align with the Fidelity International High Dividend Index. The fund has no shareholder fees and an annual operating expense of 0.18%, which includes a management fee of 0.18% and no distribution or other expenses. The fund's prospectus and additional information are available online or by contacting Fidelity.

The document provides an example illustrating the impact of fees and expenses on investments, noting that actual fees, expenses, and returns may vary. For a $10,000 investment, total expenses upon selling shares are $18 after 1 year, $58 after 3 years, $101 after 5 years, and $230 after 10 years. The fund's portfolio turnover rate was 67% of its average portfolio value in the most recent fiscal year, indicating potential higher transaction costs and tax implications. The fund invests at least 80% of its assets in securities from the Fidelity Internation

## 2. Evaluate a role play transcript: fact checking + custom evaluators 

In [6]:
transcript = """{
  "conversation": [
    {
      "speaker": "Advisor",
      "text": "Hey Jonathan, it's Alex from Fidelity Investments. How's it going today?"
    },
    {
      "speaker": "Client",
      "text": "Hey Alex, doing well, thanks! Just trying to get a better handle on some investment options. Heard you guys have some interesting products."
    },
    {
      "speaker": "Advisor",
      "text": "Absolutely, we've got a few that might catch your interest. Let's dive in. What are you curious about?"
    },
    {
      "speaker": "Client",
      "text": "Well, I'm looking for something with good returns but not too risky. I've got a decent mix already, but I'm open to exploring more. What do you suggest?"
    },
    {
      "speaker": "Advisor",
      "text": "Got it. Let's start with the Fidelity International High Dividend ETF, or FIDI. It's all about those high dividends from large and mid-cap international companies. The annual operating expense is just 0.18%, so it's pretty cost-effective."
    },
    {
      "speaker": "Client",
      "text": "Sounds interesting. How's the performance been lately?"
    },
    {
      "speaker": "Advisor",
      "text": "From 2019 to 2023, returns have ranged from -11.65% to 17.32%. In 2023, it was up 16.60% before taxes. Not too shabby, right?"
    },
    {
      "speaker": "Client",
      "text": "Yeah, not bad at all. What about risks? Anything I should be worried about?"
    },
    {
      "speaker": "Advisor",
      "text": "Good question. There's the usual market volatility and foreign exposure. Plus, since it's passively managed, it might not always match the index perfectly. But overall, it's a solid choice for dividend-focused investing."
    },
    {
      "speaker": "Client",
      "text": "Okay, cool. What else do you have?"
    },
    {
      "speaker": "Advisor",
      "text": "You might like the Fidelity Blue Chip Growth ETF, FBCG. It's all about long-term capital growth with a focus on blue chip companies. The management fee is a bit higher at 0.59%, but it's had some impressive returns."
    },
    {
      "speaker": "Client",
      "text": "Impressive, huh? Like what?"
    },
    {
      "speaker": "Advisor",
      "text": "In 2023, it returned a whopping 57.72%! But keep in mind, it's not without its ups and downs. 2022 was a rough year with a -39.01% return."
    },
    {
      "speaker": "Client",
      "text": "Wow, that's quite a swing. What's the deal with the 'Tracking Basket' thing I heard about?"
    },
    {
      "speaker": "Advisor",
      "text": "Ah, the Tracking Basket. It's a way to keep the fund's strategy under wraps, which can help protect against copycats. But it might lead to higher trading costs and price discrepancies."
    },
    {
      "speaker": "Client",
      "text": "Got it. Anything else I should consider?"
    },
    {
      "speaker": "Advisor",
      "text": "Sure, there's the Fidelity High Dividend ETF, FDVV. It's got a super low expense ratio of 0.16% and focuses on high-dividend-paying companies. The returns have been pretty steady, with a 17.90% return in 2023."
    },
    {
      "speaker": "Client",
      "text": "That sounds like a safe bet. How does it compare to the others in terms of risk?"
    },
    {
      "speaker": "Advisor",
      "text": "It's generally less volatile than the Blue Chip Growth ETF, but like any investment, there's still market risk and foreign exposure. It's a good option if you're looking for steady income."
    },
    {
      "speaker": "Client",
      "text": "Alright, thanks for breaking it down, Alex. What's the next step if I want to move forward with any of these?"
    },
    {
      "speaker": "Advisor",
      "text": "No problem, Jonathan. We can set up a time to go over your portfolio in more detail and see how these fit in. I'll send you an email with some times that work for me. Sound good?"
    },
    {
      "speaker": "Client",
      "text": "Sounds perfect. Thanks again, Alex. Looking forward to it!"
    },
    {
      "speaker": "Advisor",
      "text": "Great! Talk soon, Jonathan. Take care!"
    }
  ]
}"""

### 2.a) Fact check the call against a list of known facts about the offering

In [7]:
from wise.fact_checker import FactChecker

In [8]:
fact_string = ('##### \n').join(facts)
fact_checker = FactChecker(aoai_client, fact_string, model='gpt-4o-global')
fact_checker_report = fact_checker.check_transcript(transcript)

In [9]:
fact_checker_report

{'total_facts_shared': 18,
 'accurate_facts_count': 18,
 'fact_details': [{'fact': 'The Fidelity International High Dividend ETF (FIDI) has an annual operating expense of 0.18%.',
   'accuracy': 'accurate',
   'citation': 'The Fidelity International High Dividend ETF (FIDI) has an annual operating expense of 0.18%, which includes a management fee of 0.18% and no distribution or other expenses.'},
  {'fact': 'From 2019 to 2023, returns have ranged from -11.65% to 17.32%.',
   'accuracy': 'accurate',
   'citation': 'The document states that the performance data shows year-by-year returns: 15.84% in 2019, -11.65% in 2020, 17.32% in 2021, and -5.26% in 2022.'},
  {'fact': 'In 2023, it was up 16.60% before taxes.',
   'accuracy': 'accurate',
   'citation': 'The document states that for the period ending December 31, 2023, the Fidelity International High Dividend ETF had a return before taxes of 16.60% for the past year.'},
  {'fact': 'From 2019 to 2023, returns have ranged from -11.65% to 1

### 2.b) Use custom evaluation criteria to score the call

In [11]:
import json
from wise.evaluator import Evaluator

evaluation_criteria = [{"criteria":"Next_steps",
 "description":"The Advisor planned clear next steps for the Client to take after the conversation.",
 "score":1
 },
{"criteria":"Product_knowledge",
 "description": "The Advisor demonstrated a strong understanding of the products they were discussing.",
 "score":1
},
{
    "criteria": "Discovery",
    "description":" The Advisor asked the Client questions to understand their needs and preferences.",
    "score":1
}
]

theme = "Initial call between a wealth advisor and a prospective customer"

evaluator = Evaluator(aoai_client, evaluation_criteria, theme)
evaluation = evaluator.evaluate_transcription(transcript)

print(evaluation)

{'evaluation': {'criteria': [{'name': 'Next_steps', 'sub_criteria': [{'name': 'Clear Next Steps', 'score': 1, 'rationale': "The advisor clearly outlined the next steps by suggesting setting up a time to go over the client's portfolio in more detail and promised to send an email with available times."}], 'score': 1, 'rationale': 'The advisor effectively planned clear next steps for the client to take after the conversation.'}, {'name': 'Product_knowledge', 'sub_criteria': [{'name': 'Understanding of Products', 'score': 1, 'rationale': 'The advisor demonstrated a strong understanding of the products by providing detailed information about various investment options, including their performance, risks, and unique features.'}], 'score': 1, 'rationale': 'The advisor showed a strong understanding of the products discussed, providing comprehensive details and comparisons.'}, {'name': 'Discovery', 'sub_criteria': [{'name': 'Understanding Client Needs', 'score': 1, 'rationale': "The advisor ask